## 1. Embed the query using - 'jinaai/jina-embeddings-v2-small-en'

In [9]:
from fastembed import TextEmbedding

In [23]:
import numpy as np

In [10]:
model_name = 'jinaai/jina-embeddings-v2-small-en'

In [11]:
query = 'I just discovered the course. Can I join now?'

In [12]:
embedding_model = TextEmbedding(model_name)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

onnx/model.onnx:   0%|          | 0.00/130M [00:00<?, ?B/s]

In [13]:
embeddings_list = list(embedding_model.embed(query))

In [18]:
min(embeddings_list[0])

np.float64(-0.11726373551188797)

In [20]:
x = embeddings_list[0]

## 2. Cosine similarity

In [19]:
doc = 'Can I still join the course after the start date?'

In [21]:
doc_embedding = list(embedding_model.embed(doc))

In [22]:
y = doc_embedding[0]

In [24]:
x.dot(y)

np.float64(0.9008528856818037)

## 3. Ranking by cosine

In [25]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [30]:
highest = 0
h_index = 0
for i, item in enumerate(documents):
    text = item['text']
    e = list(embedding_model.embed(text))[0]
    sim = x.dot(e)
    print (sim)
    if sim > highest:
        highest = sim
        h_index = i

highest, h_index

0.7629684493123693
0.8182378325403556
0.8085397290762828
0.713307860072464
0.7304499388234831


(np.float64(0.8182378325403556), 1)

## 4. Cosine Version 2

In [31]:
highest = 0
h_index = 0
for i, item in enumerate(documents):
    text = item['question'] + ' ' + item['text']
    e = list(embedding_model.embed(text))[0]
    sim = x.dot(e)
    print (sim)
    if sim > highest:
        highest = sim
        h_index = i

highest, h_index

0.8514543121225202
0.8436594058703555
0.8408287206920106
0.7755157711899687
0.8086008033213788


(np.float64(0.8514543121225202), 0)

## 5. Smallest Model

In [36]:
s_model = TextEmbedding('BAAI/bge-small-en')

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/133M [00:00<?, ?B/s]

In [37]:
l = list(s_model.embed(query))[0]

In [38]:
len(l)

384

## 6. Indexing with Qdrant

In [ ]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [39]:
texts = [item['question'] + ' ' + item['text'] for i, item in enumerate(documents)]

In [41]:
m = s_model.embed(texts)

In [43]:
from qdrant_client import QdrantClient, models

client = QdrantClient(":memory:")  # Qdrant is running from RAM.

In [44]:
model_name = "BAAI/bge-small-en"
client.create_collection(
    collection_name="test_collection",
    vectors_config=models.VectorParams(
        size=client.get_embedding_size(model_name), 
        distance=models.Distance.COSINE
    ),  # size and distance are model dependent
)

True

In [48]:
metadata_with_docs = [
    {"document": item['question'] + ' ' + item['text'], "source": item['course']} for item in documents
]
ids = list(range(len(documents)))
client.upload_collection(
    collection_name="test_collection",
    vectors=[models.Document(text=doc['document'], model=model_name) for doc in metadata_with_docs],
    ids=ids,
)

In [51]:
search_result = client.query_points(
    collection_name="test_collection",
    query=models.Document(
        text=query, 
        model=model_name
    )
).points
print(search_result)

[ScoredPoint(id=0, version=0, score=0.8507920473465453, payload={}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=2, version=0, score=0.8468815448892175, payload={}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=1, version=0, score=0.8353983667389852, payload={}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=4, version=0, score=0.8297411570009187, payload={}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=3, version=0, score=0.8225325597525123, payload={}, vector=None, shard_key=None, order_value=None)]


In [52]:
max([r.score for r in search_result])

0.8507920473465453